In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),      # Resize to a common size
    transforms.ToTensor(),             # Convert to PyTorch tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to the range [-1, 1]
])


In [3]:
train_dataset = datasets.SVHN(root='./data', split='train', transform=transform, download=True)
test_dataset = datasets.SVHN(root='./data', split='test', transform=transform, download=True)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 182040794/182040794 [00:40<00:00, 4515540.78it/s]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 64275384/64275384 [00:13<00:00, 4726968.83it/s]


In [4]:
train_dataset

Dataset SVHN
    Number of datapoints: 73257
    Root location: ./data
    Split: train
    StandardTransform
Transform: Compose(
               Resize(size=(32, 32), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [5]:
# Create DataLoader for batch processing
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)  # 10 output classes for digits 0-9

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 32 * 8 * 8)  # Flatten the output from conv layers
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create an instance of the CNN
model = CNN()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print(f"Epoch [{epoch + 1}/{num_epochs}], "
                  f"Batch [{i + 1}/{len(train_loader)}], "
                  f"Loss: {running_loss / 100:.4f}")
            running_loss = 0.0

print("Training Finished!")

# Evaluation on test set
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {(correct / total) * 100:.2f}%")

Epoch [1/10], Batch [100/1145], Loss: 2.2486
Epoch [1/10], Batch [200/1145], Loss: 2.2442
Epoch [1/10], Batch [300/1145], Loss: 2.2227
Epoch [1/10], Batch [400/1145], Loss: 2.2066
Epoch [1/10], Batch [500/1145], Loss: 1.9764
Epoch [1/10], Batch [600/1145], Loss: 1.3520
Epoch [1/10], Batch [700/1145], Loss: 1.0038
Epoch [1/10], Batch [800/1145], Loss: 0.8301
Epoch [1/10], Batch [900/1145], Loss: 0.7782
Epoch [1/10], Batch [1000/1145], Loss: 0.6789
Epoch [1/10], Batch [1100/1145], Loss: 0.5896
Epoch [2/10], Batch [100/1145], Loss: 0.5696
Epoch [2/10], Batch [200/1145], Loss: 0.5628
Epoch [2/10], Batch [300/1145], Loss: 0.5133
Epoch [2/10], Batch [400/1145], Loss: 0.5163
Epoch [2/10], Batch [500/1145], Loss: 0.5032
Epoch [2/10], Batch [600/1145], Loss: 0.5070
Epoch [2/10], Batch [700/1145], Loss: 0.4738
Epoch [2/10], Batch [800/1145], Loss: 0.4784
Epoch [2/10], Batch [900/1145], Loss: 0.4610
Epoch [2/10], Batch [1000/1145], Loss: 0.4366
Epoch [2/10], Batch [1100/1145], Loss: 0.4475
Epoch 